In [1]:
import pandas as pd
from gurobipy import *
import openpyxl

In [2]:
#function of model
def exceltoGurobi(main):
    
    # read objective
    obj = main.cell(column=2, row=1).value
    if obj =='max':
        obj = -1
    elif obj =='min':
        obj = 1
    else:
        raise NameError('Objective cannot be recognized')
    # read obj coefficient
    obj_coeff=[]
    for c in range(2,10000):
        v = main.cell(column=c, row=3).value
        if v is None:
            break
        else:
            obj_coeff.append(v)
    n_variables = len(obj_coeff)
   
    #read variable properties
    variable_name = []
    for c in range(n_variables):
        v = main.cell(column=c+2, row=2).value
        if v is None:
            variable_name.append('x'+str(c+1))
        elif type(v) != str:
            variable_name.append(str(v))
        else:
            variable_name.append(v)        
        
    variable_type = []
    for c in range(n_variables):
        v = main.cell(column=c+2, row=4).value
        if v is None:
            t = 'C' 
        elif v == 'cont':
            t = 'C'
        elif v == 'int':
            t = 'I'
        elif v == 'bin':
            t = 'B'
        else:
            raise NameError('Variable type cannot be recognized')
        variable_type.append(t)
    
    variable_lb = []
    for c in range(n_variables):
        v = main.cell(column=c+2, row=5).value
        if v is None:
            t = 0
        elif v == '"+inf"':
            raise NameError('Variable lower bound cannot be +inf')
        elif v == '"-inf"':
            t = -GRB.INFINITY
        elif type(v) in [float, int]:
            t = v
        else:
            raise NameError('Variable lower bound cannot be recognized')
        variable_lb.append(t)

    variable_ub = []
    for c in range(n_variables):
        v = main.cell(column=c+2, row=6).value
        if v is None:
            t = GRB.INFINITY
        elif v == '"+inf"':
            t = GRB.INFINITY
        elif v == '"-inf"':
            raise NameError('Variable upper bound cannot be -inf')
        elif type(v) in [float, int]:
            t = v
        else:
            raise NameError('Variable upper bound cannot be recognized')
        variable_ub.append(t)
        
        
        
    # read constraints
    ## find constraint column index
    for c in range(1, n_variables+100):
        v = main.cell(column=c, row=8).value
        if v == 'constraint type':
            const_typ_cind = c
            break

    if const_typ_cind-2 != n_variables:
        raise NameError('Number of variables does not match the constraints')
    
    ## find last constraint row index
    for r in range(9, 100000):
        v = main.cell(column=const_typ_cind, row=r).value
        if v is None:
            break
        elif v not in ["<=", ">=", "="]:
            raise NameError("Wrong constraint type")
        else:
            last_r_consraints=r

    ## read constraints as dataframe
    for r in range(9, last_r_consraints+1):
        v = main.cell(column=const_typ_cind+2, row=r).value
        if v is None:
            main.cell(column=const_typ_cind+2, row=r).value = "c"+str(r-8)
        elif type(v) != str:
            main.cell(column=const_typ_cind+2, row=r).value = str(v)
        else:
            1==1        
    values = []
    cons_df = pd.DataFrame()
    for r in range(9, last_r_consraints+1):
        r_values = []
        for c in range(2, const_typ_cind+3):
            r_values.append(main.cell(column=c, row=r).value)
        values.append(r_values)
        cons_df = pd.DataFrame(values)
        cons_df.fillna(0, inplace=True)
        cons_df.rename(columns={const_typ_cind-2:'constraint type', 
                                const_typ_cind-1:'RHS values', 
                                const_typ_cind:'constraint name'}, inplace=True)
    # setup model
    m = Model()
    x=m.addVars(n_variables)
    # set types, lb, ub of variables
    for i in range(n_variables):
        x[i].setAttr('VarNAME', variable_name[i])
        x[i].setAttr('vType', variable_type[i])
        x[i].setAttr('lb', variable_lb[i])
        x[i].setAttr('ub', variable_ub[i])
    # set objective
    objective = quicksum(obj_coeff[i] * x[i] for i in range(n_variables))
    m.setObjective(objective, obj)
    # add constraints
    if cons_df.shape[0]:
        ## add le constraints
        le_const = cons_df[cons_df['constraint type'] == '<=']
        le_const.reset_index(inplace=True, drop=True)
        for i in range(le_const.shape[0]):
            m.addConstr(quicksum(le_const.iloc[i,j]* x[j] for j in range(n_variables)) <= le_const['RHS values'][i], 
                        name = le_const['constraint name'][i])
        ## add ge constraints
        ge_const = cons_df[cons_df['constraint type'] == '>=']
        ge_const.reset_index(inplace=True, drop=True)
        for i in range(ge_const.shape[0]):
            m.addConstr(quicksum(ge_const.iloc[i,j]* x[j] for j in range(n_variables)) >= ge_const['RHS values'][i], 
                        name = ge_const['constraint name'][i])
        ## add eq constraints
        eq_const = cons_df[cons_df['constraint type'] == '=']
        eq_const.reset_index(inplace=True, drop=True)
        for i in range(eq_const.shape[0]):
            m.addConstr(quicksum(eq_const.iloc[i,j]* x[j] for j in range(n_variables)) == eq_const['RHS values'][i], 
                        name = eq_const['constraint name'][i])
    m.optimize()
    print("The best solution's profit is "+str(m.objVal))
    m.printAttr('X')
    
    
    
    # sensitivity analysis
    for v in m.getVars():

        print("For Variable " + v.VarName+ ":")
        print("Minimum value coefficient can take before the optimal decision changes "  + "is " + str(v.SAObjLow))
        print("Maximum value coefficient can take before the optimal decision changes "  + "is " + str(v.SAObjUp))
    
    for c in m.getConstrs():
        print("For constraint " + c.ConstrName+ ":")
        print("Shawdow Price is " + str(c.pi))
        print("Minimum value RHS can take before the shadow price changes "  + "is " + str(c.SARHSLow))
        print("Maximum value RHS can take before the shadow price changes "  + "is " + str(c.SARHSUp))
    

### 1. Anderson Electronics 
### a)

In [3]:
# reading the inputs from Excel workbook
wb = openpyxl.load_workbook("Group8_HW2.xlsx")
main = wb['Main']
exceltoGurobi(main)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-04
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 3 rows, 4 columns and 12 nonzeros
Model fingerprint: 0x3f5598fd
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [3e+01, 7e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+03, 5e+03]
Presolve time: 0.01s
Presolved: 3 rows, 4 columns, 12 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.8700000e+32   8.000000e+30   1.870000e+02      0s
       3    6.9400000e+04   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds (0.00 work units)
Optimal objective  6.940000000e+04
The best solution's profit is 69400.00000000001

    Variable            X 
-------------------------
           S          380 
           B         1060 
For Variable D:
Minimum

### The optimal solution is produce 380 Stereo and 1060 Blue Ray players. The expected profit is $69400.

### b) According to the sensitivity analysis, if the profit of Blue Ray players is between 48.99 to 64, which means the selling price is  between 104.99 to 120, current production would be optimal. So if Blue Ray players sold for 106, Anderson’s total profit be  (106-56)$\times$1060+380$\times$32=65160 

### c) No

### d) According to the sensitivity analysis, the shadow price of electronic components is 2 dollars. It means from 3750 to 7499.9,each additional electronic component will increase the profit by 2 dollars. If we could increase the supply of electronic components by 400 units, the total amount of electronic components is 5100, so the profit will increase 400$\times$2=800 dollars.  If we could increase the supply of electronic components by 4000 units, the total amount of electronic components is 8700, the optimal production is as follows:

In [4]:
# reading the inputs from Excel workbook
wb = openpyxl.load_workbook("Group8_HW2.xlsx")
main_d = wb['Main-d']
exceltoGurobi(main_d)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 3 rows, 4 columns and 12 nonzeros
Model fingerprint: 0x3c87c9cf
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [3e+01, 7e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+03, 9e+03]
Presolve time: 0.01s
Presolved: 3 rows, 4 columns, 12 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.8700000e+32   8.000000e+30   1.870000e+02      0s
       4    7.5000000e+04   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  7.500000000e+04
The best solution's profit is 75000.0

    Variable            X 
-------------------------
           S         1500 
           B          500 
For Variable D:
Minimum value coefficient can take before the optimal decision changes is -inf
Maximum value coefficient can

### e) If supplier would charge 8 dollars/unit for 400 extras of electronic components, the objective function and constraints would change. New optimal production is as follows:

In [5]:
# reading the inputs from Excel workbook
wb = openpyxl.load_workbook("Group8_HW2.xlsx")
main_e = wb['Main-e']
exceltoGurobi(main_e)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 4 rows, 4 columns and 16 nonzeros
Model fingerprint: 0x90c1ef92
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [3e+01, 7e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+03, 5e+03]
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 3 rows, 5 columns, 13 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.6700000e+04   1.007617e+02   0.000000e+00      0s
       2    6.5100000e+04   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.00 seconds (0.00 work units)
Optimal objective  6.510000000e+04
The best solution's profit is 65100.0

    Variable            X 
-------------------------
           S          540 
           B          980 
For Variable D:
Minimum value coefficient can take before the optimal decision changes

### f)  Same as question e

In [6]:
# reading the inputs from Excel workbook
wb = openpyxl.load_workbook("Group8_HW2.xlsx")
main_f = wb['Main-f']
exceltoGurobi(main_f)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 4 rows, 4 columns and 16 nonzeros
Model fingerprint: 0x2a49c87e
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [2e+01, 6e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+03, 5e+03]
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 3 rows, 5 columns, 13 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.4250000e+04   8.497060e+02   0.000000e+00      0s
       2    6.1650000e+04   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  6.165000000e+04
The best solution's profit is 61650.0

    Variable            X 
-------------------------
           S          230 
           B         1260 
For Variable D:
Minimum value coefficient can take before the optimal decision changes

### 2. Lawsuit Cash Flow Problem

In [7]:
wb = openpyxl.load_workbook("Group8_HW2.xlsx")
main2 = wb['Main2']
exceltoGurobi(main2)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 14 rows, 17 columns and 45 nonzeros
Model fingerprint: 0xb397b704
Coefficient statistics:
  Matrix range     [6e-02, 1e+00]
  Objective range  [1e-08, 1e+00]
  Bounds range     [1e+01, 4e+01]
  RHS range        [1e+01, 3e+01]
Presolve removed 2 rows and 2 columns
Presolve time: 0.01s
Presolved: 12 rows, 15 columns, 41 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000004e+01   3.812685e+01   0.000000e+00      0s
      14    1.9568352e+02   0.000000e+00   0.000000e+00      0s

Solved in 14 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.956835243e+02
The best solution's profit is 195.68352431720746

    Variable            X 
-------------------------
          x0      93.8789 
          y0      86.9991 
          z0      14.8055 
          z1      16.6053 
          z2    